# Mycologue Labs - Image to 3D workflow

The following workflow demonstrates how to turn mulitple directories of images of a mushroom specimen into a high fidelity 3D models.

This workflow does not include the processing of RAW images to color corrected JPG images, which is highly recommended for quality 3D model textures.

In [ ]:
# define the locations of each image directory to process
input_dirs = ['photos/20241011-1408-admirable-bolete']

# import os
# input_dirs = [os.path.join('photos', d) for d in os.listdir('photos') if os.path.isdir(os.path.join('photos', d))]

In [ ]:
# generate masks for images
for input_dir in input_dirs:
    !python create-masks.py --input {input_dir}

In [ ]:
# create realityscan project and export 3d model
for input_dir in input_dirs:
    !python create-models.py --input {input_dir}